#### Outputs a **.csv** file with metrics on repositories from the following organizations:
- googlesamples
- aws-samples
- Azure-Samples
- spring-guides 
- googlearchive
- spring-cloud-samples
-spring-io
#### The metrics include:
- full_name
- created_at
- description
- forks_count
- language
- open_issues_count
- size
- stargazers_count
- subscribers_count
- updated_at
- watchers_count
- langs_percentage
x contibutors count
x issues and pull reqs open and closed
x accepted pull reqs?
x num of commits
x num of branches
x last and first commit dates in the .csv file
x tags
<div class="alert alert-box alert-info">
    <b>Note:</b> You can change the organizations by modifying the list of organizations in the code cell below.
</div>

In [1]:
%pip install PyGithub
%pip install python-dotenv
%pip install pandas
%pip install tqdm
%pip install cachetools

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: C:\Users\oheit\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: C:\Users\oheit\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: C:\Users\oheit\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: C:\Users\oheit\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: C:\Users\oheit\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [2]:
from github import Github
import pandas as pd
from dotenv import load_dotenv
from os import getenv
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm
from cachetools import cached, TTLCache
import time

In [3]:
load_dotenv()
g = Github(getenv('GITHUB_TOKEN'))

In [4]:
cache = TTLCache(maxsize=1024, ttl=300)

@cached(cache)
def fetch_repo_data(repo):
    time.sleep
    all_issues = list(repo.get_issues(state="all"))
    open_issues_count = sum(1 for issue in all_issues if issue.state == "open")
    closed_issues_count = len(all_issues) - open_issues_count
    
    all_pulls = list(repo.get_pulls(state='all'))
    open_pulls_count = sum(1 for pull in all_pulls if pull.state == "open")
    closed_pulls_count = len(all_pulls) - open_pulls_count
    merged_pulls_count = sum(1 for pull in all_pulls if pull.merged)
    
    commits = list(repo.get_commits())
    commits_count = len(commits)
    first_commit_date = commits[-1].commit.author.date if commits else None
    last_commit_date = commits[0].commit.author.date if commits else None

    return {
        "issues_count": len(all_issues),
        "closed_issues_count": closed_issues_count,
        "open_issues_count": open_issues_count,
        "total_issues_count": closed_issues_count + open_issues_count,
        "closed_pulls_count": closed_pulls_count,
        "open_pulls_count": open_pulls_count,
        "total_pulls_count": closed_pulls_count + open_pulls_count,
        "merged_pulls_count": merged_pulls_count,
        "commits_count": commits_count,
        "first_commit_date": first_commit_date,
        "last_commit_date": last_commit_date,
        "branches_count": int(repo.get_branches().totalCount),
        "topics": str(tuple(repo.topics)),
        "contributors_count": int(repo.get_contributors().totalCount)
    }

def get_repo_data(full_name):
    repo = g.get_repo(full_name)
    return fetch_repo_data(repo)

In [5]:

def fetch_all_repo_data(full_names):
    data_list = []
    
    with ThreadPoolExecutor(max_workers=20) as executor:
        future_to_repo = {executor.submit(get_repo_data, full_name): full_name for full_name in full_names}
        for future in tqdm(as_completed(future_to_repo), total=len(full_names), unit="repo", ncols=100, bar_format="{l_bar}{bar}| {n_fmt}/{total_fmt} [{elapsed}<{remaining}, {rate_fmt}{postfix}] {percentage:3.0f}%"):
            try:
                data = future.result()
                full_name = future_to_repo[future]
                data["full_name"] = full_name
                data_list.append(data)
            except Exception as e:
                print(f"Error fetching data for repo: {future_to_repo[future]}, error: {e}")

    return pd.DataFrame(data_list)

In [6]:
dataframe = pd.read_csv('codesamples.csv')
full_names = dataframe['full_name'].unique()

repo_data_df = fetch_all_repo_data(full_names)

dataframe = pd.merge(dataframe, repo_data_df, on='full_name', how='left')

  0%|                                                               | 0/7759 [00:00<?, ?repo/s]   0%Request GET /repos/googlesamples/android-media-controller failed with 403: Forbidden
Setting next backoff to 113.952854s
Request GET /repos/googlesamples/oauth-apps-for-windows failed with 403: Forbidden
Request GET /repos/googlesamples/android-AppUsageStatistics failed with 403: Forbidden
Request GET /repos/googlesamples/android-vision failed with 403: Forbidden
Setting next backoff to 113.913572s
Request GET /repos/googlesamples/androidtv-sample-inputs failed with 403: Forbidden
Request GET /repos/googlesamples/unity-jar-resolver failed with 403: Forbidden
Request GET /repos/googlesamples/android-custom-lint-rules failed with 403: Forbidden
Setting next backoff to 113.904531s
Request GET /repos/googlesamples/android-vulkan-tutorials failed with 403: Forbidden
Request GET /repos/googlesamples/google-services failed with 403: Forbidden
Request GET /repos/googlesamples/brillo-dragonboard-